Create another model that includes age in addition to fare and pclass. Does this model perform better than your previous one?

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from acquire import get_titanic_data

import warnings
warnings.filterwarnings("ignore")

In [6]:
titanic = get_titanic_data()
titanic.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [8]:
# removing duplicates and null values
df = titanic
df = df[~ df.age.isna()]
df = df[~ df.embarked.isna()]
df = df.drop_duplicates()
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [9]:
df['target_var'] = df.survived

cols_to_drop = ['deck','embark_town','embarked','class']
df.drop(columns=cols_to_drop, inplace = True)
df['inputed_age'] = df.age
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,alone,target_var,inputed_age
0,0,0,3,male,22.0,1,0,7.2500,0,0,22.0
1,1,1,1,female,38.0,1,0,71.2833,0,1,38.0
2,2,1,3,female,26.0,0,0,7.9250,1,1,26.0
3,3,1,1,female,35.0,1,0,53.1000,0,1,35.0
4,4,0,3,male,35.0,0,0,8.0500,1,0,35.0
